In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 43.8MB/s 
     |████████████████████████████████| 92kB 8.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
import re
import nltk
import os
from konlpy.tag import Okt
from konlpy.tag import Komoran
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from shutil import rmtree


nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise


def del_folder(path):
    try:
        rmtree(path)
    except:
        pass


In [5]:

BASE_DIR = "/content/gdrive/My Drive/Colab Notebooks/ETRI_Article_Summarizer/Text-preprocessing-Data/articles"
ORIGIN_PATH = os.path.join(BASE_DIR,"Origin-Data")
PREPROCESSED_PATH = os.path.join(BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(BASE_DIR,"Pretty-Data")
SWORDS_PATH = os.path.join(BASE_DIR, "StopWordList.txt")


In [19]:
class TextPreprocessor:
    def __init__(self):
        self.retokenize = RegexpTokenizer("[\w]+")
        self.swords = []
        self.tokenizer = {}
        self.tagger = Komoran()

    def removeDuplicateSpace(self, text):
        return re.sub('\s+', ' ', text)  # 중복 공백, 탭, 개행 제거
    
    def removeSpecialChar(self, text):
        return ' '.join(self.retokenize.tokenize(text))


    def loadSwords(self, filename):
        self.swords = []
        with open(filename, 'r') as f:
            swords = f.readlines()
            self.swords = [tag for sword in self.swords for tag in self.tagger.pos(sword.strip()) if tag[1] in ('NNG', 'NNP', 'VV', 'VA')]

        self.tokenizer = lambda sent: filter(lambda x:x not in self.swords and x[1] in ('NNG', 'NNP', 'VV', 'VA'), self.tagger.pos(sent))

        return self.swords
        
    def removeSwords(self, text):
        return ' '.join([noun for (noun, pos) in list(self.tokenizer(text))]) 

In [10]:
def saveTextFile(baseDir, media, filename, sentences):

    mkdir_p(os.path.join(baseDir, media))
    save_path = os.path.join(os.path.join(baseDir, media), filename)

    with open(save_path, 'w') as f:
        f.write('/n'.join([sentence for sentence in sentences if sentence is not '']))


In [20]:
class Article:
    def __init__(self, articleInfo):
        self.title = articleInfo[0]
        self.media = articleInfo[1]
        self.content = articleInfo[2:]

    def readContent(self):
        for line in self.content:
            if line is '': continue
            yield line            

In [30]:
class ArticleReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
        self.content = ''

    def __iter__(self):
        with open(self.filepath, encoding='utf-8') as f:
            title = f.readline()[:-1]
            yield title
            self.content = f.readline()[:-1]

            media = f.readline()[:-1]
            yield media

            self.del_personal_info(media)
            docs = self.rgxSplitter.split(self.content)
            for s in map(lambda a, b: a + b, docs[::2], docs[1::2]):
                if not s: continue
                yield s

    def del_personal_info(self, media):
        rmBracket = re.sub('(\([^)]*\)|\[[^]]*\])', '', self.content)  # 괄호 안 내용 제거
        rmMedia = re.sub(media, ' ', rmBracket)  # 언론사명 제거
        rmReporter = re.sub('[가-힣]{2,5}\s?기자', ' ', rmMedia) # 기자 이름 제거
        rmEmail = re.sub('[0-9a-zA-Z]([-_\.]?[0-9a-zA-Z])*@[0-9a-zA-Z]([-_\.]?[0-9a-zA-Z])*\.[a-zA-Z]{2,3}', ' ', rmReporter) # 이메일 제거

        self.content = rmEmail


In [32]:
del_folder(PREPROCESSED_PATH)
del_folder(PRETTY_PATH)

preprocessor = TextPreprocessor()
preprocessor.loadSwords(SWORDS_PATH)

if __name__ == '__main__':
    media_list = os.listdir(ORIGIN_PATH)

    for media in media_list:

        media_path = os.path.join(ORIGIN_PATH, media)
        article_list = os.listdir(media_path)

        for article_name in article_list:
            
            reader = ArticleReader(os.path.join(media_path, article_name))
            article = Article(list(filter(None, reader)))

            prettyLine = []
            preprocessedLine = []
            for line in article.readContent() :
                cleanLine = preprocessor.removeDuplicateSpace(line)
                cleanLine = preprocessor.removeSpecialChar(cleanLine)
                
                rmSwordLine = preprocessor.removeSwords(cleanLine)

                preprocessedLine.append(rmSwordLine)

            saveTextFile(PREPROCESSED_PATH, media, article_name, preprocessedLine)
            saveTextFile(PRETTY_PATH, media, article_name, article.readContent())

“정인이 멍 외면한 홀트·아동기관…양모 위로한 경찰” 국민일보
고개숙인 경찰청장 국민일보
“정인이 사건 방조한 경찰 파면하라” 靑청원 20만 돌파 국민일보
경찰 간부, 방역수칙 어기고 노래방서 지인들과 "술판" TV조선
경찰청장, "정인이 사건" 대국민 사과…"양천서장 대기발령" SBS
경찰, 지적장애 여성 성추행한 BJ 긴급체포 SBS
"차에 개 매달고 5km 주행" 50대 경찰 조사 YTN
"정인이 사건" 경찰청장 대국민 사과...양천서장 대기발령 YTN
경찰청장 "새로운 증거 없인 양부모 살인죄 적용 어렵다" YTN
제주신화월드 카지노에서 145억 원 도난...경찰 수사 YTN
차에서 내린 시민들..."제설도 경찰 도움도 없었다" YTN
충북 경찰 4년 연속 총경 2명 배출 뉴스1
당정, 아동학대 현장 경찰 출입·조사권 확대…긴급대책 내주 발표 뉴스1
미 경찰 "국회의사당 구내서 4명 사망…52명 체포" 뉴스1
장흥군, 코로나19 예방 위한 "자치경찰 TF팀" 시범운영 뉴스1
정인이 못 살린 양천경찰서만 그럴까?…"경찰과 협조 안돼" 뉴스1
"정인이 생명 보호하지 못했다"…경찰청장 사과 MBC
"신고 묵살" 경찰 질타…중대재해법 "초읽기" MBC
경찰, "영진약품 부당합병 의혹" KT&G 압수수색 MBC
신화월드 카지노 횡령 사건, 제주경찰청이 수사 MBC
금은방 털이한 현직 경찰관…범행 20일 만에 체포 노컷뉴스
"학대신고 3번이면 구속"…7년 전 경찰 대책은 어디로? 노컷뉴스
울산경찰청 원용덕·양순봉 계장 총경 승진 노컷뉴스
정인이‧이용구‧박원순까지 "부실"…안개 낀 경찰의 "책임수사" 노컷뉴스
사라진 145억원…제주경찰, 50대 여직원 행방 추적중 노컷뉴스
광주전남 "경찰의 꽃" 총경 7명 승진 노컷뉴스
경찰청장 ‘정인이 사건’ 공식 사과…양천서장 대기발령 경향신문
홍콩 경찰, 범민주 인사 53명 체포 경향신문
부산경찰청, 경무관 승진자 2명 첫 배출 KBS
특별방역기간 불 꺼진 노래방서 술판 벌인 경찰 간부 MBN
"수사 미흡했다"…경찰청장 사과에